# Load the weights files from the folder ../weights

In [1]:
from sklearn.preprocessing import normalize
import numpy as np

# Substitute the weights file you have just generated
weights_CR = np.load("./weights/weights_CR.npy")
weights_PFC = np.load("./weights/weights_PFC.npy")
weights_VC = np.load("./weights/weights_VC.npy")

# Substitute the xgboost model you have generated
# indexes_xgboost = np.load("./weights/indexes_xgboost.npy")

In [49]:
data_VC['ID'][:,1]

import pandas as pd

annotation = pd.read_csv('../annotation.csv')

In [162]:
ro_id = annotation.ix[:,1].values
entrez = annotation.ix[:,2].values
genbank = annotation.ix[:,4].values

In [210]:
mappingGenbank = {ro_id[i]:genbank[i] for i in xrange(39302)}
mappingEntrez = {ro_id[i]:entrez[i] for i in xrange(39302)}

In [82]:
genbank_list = np.array([mapping[i] for i in data_VC['ID'][:,1]]).astype(str)

In [106]:
index_nonNA = np.array(np.where(genbank_list!='nan'))[0]

In [114]:
index_nonNA.shape

(37138,)

In [90]:
genbank_nonNA_list = genbank_list[genbank_list!='nan']

In [93]:
# for i in genbank_nonNA_list:
#     print i

In [116]:
temp = np.concatenate((index_nonNA.reshape(37138,1),genbank_nonNA_list.reshape(37138,1)),axis=1)

In [117]:
temp.shape

(37138, 2)

In [120]:
dataframe = pd.DataFrame(temp)

dataframe.to_csv("genebank_list_nonNA.csv")

# Load the data

In [225]:
from scipy import io as sio
from sklearn.preprocessing import Imputer

data_PFC = sio.loadmat("/Volumes/TONY/LabWW/Alzheimer/GSE44772/GSE44772_PFC.mat")
data_VC = sio.loadmat("/Volumes/TONY/LabWW/Alzheimer/GSE44772/GSE44772_VC.mat")
data_CR = sio.loadmat("/Volumes/TONY/LabWW/Alzheimer/GSE44772/GSE44772_CR.mat")

def setRegion(region):
    if region == 'PFC':
        ourdata = data_PFC
    elif region == 'VC':
        ourdata = data_VC
    elif region == 'CR':
        ourdata = data_CR
    elif region == 'All':
        ourdata = data_All
        
    inputX = ourdata['X']
    # Inpute using median
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp.fit(inputX)
    inputX = imp.transform(inputX)
    
    inputY = ourdata['y'][0,:]
    
    ID = ourdata['ID'][:,1]
    
    return inputX, inputY, ID

In [275]:
weights_dfs = weights_CR
inputX, inputY, ID = setRegion('CR')
IG = sio.loadmat('../data/CR_IG.mat')
ig = IG['IG'][0]

best, index_best = unionDFSfeatures(9)
best_entrez = np.array([mappingEntrez[ID[i]] for i in index_best]).astype(int)
best_genbank = np.array([mappingGenbank[ID[i]] for i in index_best])

In [276]:
refinedBest = list(np.array(index_best)[list(np.where(best_entrez>0)[0])])

In [277]:
for i in best_entrez[best_entrez>0]:
    print i

113146
10763
7432
6363
1601
153688
1376
604
8974
54898
5179
4311
1984
59352
2205
399563
3151
4986
4311
4311
57214
3614
2662
1815
4192
11197
84873
145873
414
54716
2557
113278
116238
284800
89122
285704
83447
57722
55536
5015
79144
81625
6436
169611
120425
55415
286410
10418
254295
3250
5212
3479
340419
91612
153328
25907
202151
3704
27152
79644
64072
6387
5650
28231
29930
89122
129790
80174
112609
10590
83872
56061
8074
124404
57125
221914
5935
10521
389833
388439
84963
390079
284801
286177
441733
387648
493860
283029
152078
126537
253017
441932
440885
389336
3117
771
440138
440993
57817
28231
3250
6822


In [283]:
np.mean(cross_val_score(svm, inputX[:, refinedBest], inputY, cv=5))

0.98685990338164253

In [282]:
len(refinedBest)

102

In [269]:
len(refinedBest)

138

# Functions that regenerate the input data X based on different standards

In [206]:
# Averaging the weights based on the sum of all the normalized weights
averagedWeight = weights_dfs.sum(axis=0)
# ranking indexes for DFS (Average)
indexes_average_dfs = np.argsort(averagedWeight)[::-1]

# Select features in rank [a, b] from average DFS model, and also the indexes of these features
def topAveDFSfeatures(a,b):
    return inputX[:, indexes_average_dfs.tolist()[a:b]], indexes_average_dfs.tolist()[a:b]

# Select the union of top N features from each of the DFS model, and also the indexes of these features
def unionDFSfeatures(n):
    indexes_union = []
    for i in xrange(30):
        indexes_union.append(np.argsort(weights_dfs[i])[::-1][:n].tolist())
        
    union = reduce(np.union1d, indexes_union).tolist()
#     print("Number of union features:", len(union))
    
    return inputX[:, union], union

# Select the intersection of top N features from each of the DFS model, and also the indexes of these features
def intersectDFSfeatures(n):
    indexes_intersect = []
    for i in xrange(30):
        indexes_intersect.append(np.argsort(weights_dfs[i])[::-1][:n].tolist())
    
    intersected = reduce(np.intersect1d, indexes_intersect).tolist()
#     print("Number of intersected features:",len(intersected))
    return inputX[:, intersected], intersected

# Select features in rank [a, b] from each of the XGBoost model, and also the indexes of these features
def topXGBoostfeatures(a,b):
    return inputX[:, indexes_xgboost.tolist()[a:b]], indexes_xgboost.tolist()[a:b]

# SSelect features in rank [a, b] from one of the DFS model (n stands for which single model), 
# and also the indexes of these features
def pickOneDFSfeatures(a,b,n):
    indexx = np.argsort(weights_dfs[n])[::-1]
    return inputX[:, indexx.tolist()[a:b]], indexx.tolist()[a:b]

In [274]:
# Run SVM with top features from DFS union model
scoresUnion = []
n_unions = []
top_n = []
for i in xrange(1,20):
    print i
    keke, union = unionDFSfeatures(i)
    if len(n_unions) == 0 or len(union) != n_unions[-1]:
        top_n.append(i)
        n_unions.append(len(union))
        scoresUnion.append(np.mean(cross_val_score(svm, keke, inputY, cv=5)))
        print("Union:", scoresUnion[-1])
    
traceUnion = go.Scatter(
    x = n_unions,
    y = scoresUnion,
    mode = 'lines+markers',
    name = 'DFS Union'
)

print("DFS Union has been ran.")

"""
    Below models are ran with maximu n == n_unions[-1]
"""

# Run SVM with top features from DFS average model
scoresAve = []
for n in xrange(1, n_unions[-1]):
    temp, _ = topAveDFSfeatures(0,n)
    scoresAve.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("Ave:", scoresAve[-1])

traceAve = go.Scatter(
    x = range(1, n_unions[-1]),
    y = scoresAve,
    mode = 'lines+markers',
    name = 'DFS Ave'
)

print("DFS average has been ran.")

# Run SVM with top features from DFS average model
scoresIG = []
for n in xrange(1, n_unions[-1]):
    temp, _ = topIGfeatures(0,n)
    scoresIG.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("IG:", scoresIG[-1])

traceIG = go.Scatter(
    x = range(1, n_unions[-1]),
    y = scoresIG,
    mode = 'lines+markers',
    name = 'IG'
)

print("IG has been ran.")

# Run SVM with top features from DFS single models
tracesSingle = []
for i in xrange(5):
    print("Has ran a single DFS model:"+str(i))
    scoresNormal = []
    for n in xrange(1, n_unions[-1]):
        temp, _ = pickOneDFSfeatures(0,n,i)
        scoresNormal.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
        
    trace = go.Scatter(
        x = range(1, n_unions[-1]),
        y = scoresNormal,
        mode = 'lines+markers',
        name = 'DFS Single: '+str(i)
    )
    tracesSingle.append(trace)
    
# scoresXgboost = []
# for n in xrange(1, n_unions[-1]):
#     temp, _ = topXGBoostfeatures(0,n)
#     scoresXgboost.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
#     print("XGBoost:", scoresXgboost[-1])

# traceXGB = go.Scatter(
#     x = range(1, n_unions[-1]),
#     y = scoresXgboost,
#     mode = 'lines+markers',
#     name = 'lines+markers'
# )

# print("XGBoost model has been ran.")

1
('Union:', 0.89194367355329418)
2
('Union:', 0.95670264158700802)
3
('Union:', 0.94800698941309491)
4
('Union:', 0.9653982937609209)
5
('Union:', 0.96095796073594408)
6
('Union:', 0.97400143899681368)
7
('Union:', 0.978160139788262)
8
('Union:', 0.978160139788262)
9
('Union:', 0.98241545893719806)
10
('Union:', 0.98241545893719806)
11
('Union:', 0.97825675814574975)
12
('Union:', 0.96955699455236921)
13
('Union:', 0.97834926508377007)
14
('Union:', 0.96974611984787751)
15
('Union:', 0.96964950149038953)
16
('Union:', 0.96964950149038953)
17
('Union:', 0.974093945934834)
18
('Union:', 0.96095384931647643)
19
('Union:', 0.96095384931647643)
DFS Union has been ran.
('Ave:', 0.55199095487717131)
('Ave:', 0.54764312879021482)
('Ave:', 0.55208757323465929)
('Ave:', 0.70835234864837093)
('Ave:', 0.68661321821358823)
('Ave:', 0.69501901531503751)
('Ave:', 0.70778497276184604)
('Ave:', 0.6816939048206393)
('Ave:', 0.69493061979648474)
('Ave:', 0.69067530064754856)
('Ave:', 0.71280090451228284

In [143]:
best, index_best = unionDFSfeatures(7)

In [147]:
data_VC['ID'][i]

array([        780, 10023806539])

In [260]:
a = [traceUnion,traceAve, traceIG]
a.extend(tracesSingle)

py.iplot(a, filename='plot')

# Compare features [0,10], [10,20], [20,30] ... [90,100]

In [202]:
# scoresXgboost_range = []
# for i in xrange(20):
#     temp, _ = topXGBoostfeatures(i,i+10)
#     scoresXgboost_range.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
#     print("XGBoost:", scoresXgboost_range[-1])

# traceXGB_range = go.Scatter(
#     x = range(20),
#     y = scoresXgboost_range,
#     mode = 'lines+markers',
#     name = 'lines+markers'
# )

scoresAve_range = []
for i in xrange(20):
    temp, _ = topAveDFSfeatures(i,i+50)
    scoresAve_range.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("Ave:", scoresAve_range[-1])

traceAve_range = go.Scatter(
    x = range(20),
    y = scoresAve_range,
    mode = 'lines+markers',
    name = 'DFS Ave'
)

scoresIG_range = []
for i in xrange(20):
    temp, _ = topIGfeatures(i,i+50)
    scoresIG_range.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("IG:", scoresIG_range[-1])

traceIG_range = go.Scatter(
    x = range(20),
    y = scoresIG_range,
    mode = 'lines+markers',
    name = 'IG'
)

# Run SVM with top features from DFS single models
tracesSingle_range = []
for i in xrange(5):
    print("Has ran a single DFS model:"+str(i))
    scoresNormal = []
    for j in xrange(20):
        temp, _ = pickOneDFSfeatures(j,j+50,i)
        scoresNormal.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
        
    trace = go.Scatter(
        x = range(20),
        y = scoresNormal,
        mode = 'lines+markers',
        name = 'DFS Single: '+str(i)
    )
    tracesSingle_range.append(trace)

('Ave:', 0.90867509507657507)
('Ave:', 0.90867509507657507)
('Ave:', 0.90867509507657507)
('Ave:', 0.90867509507657507)
('Ave:', 0.90877171343406304)
('Ave:', 0.91728235173193551)
('Ave:', 0.92162606639942446)
('Ave:', 0.92162606639942446)
('Ave:', 0.91737074725048839)
('Ave:', 0.89128379072874908)
('Ave:', 0.90433138040908623)
('Ave:', 0.90442388734710666)
('Ave:', 0.91737485866995583)
('Ave:', 0.90867920649604272)
('Ave:', 0.89998355432212984)
('Ave:', 0.90442388734710666)
('Ave:', 0.89138040908623706)
('Ave:', 0.89138040908623706)
('Ave:', 0.89128790214821674)
('Ave:', 0.89147291602425738)
('IG:', 0.90432726898961879)
('IG:', 0.90867509507657507)
('IG:', 0.90432726898961868)
('IG:', 0.90877171343406304)
('IG:', 0.89563161681570558)
('IG:', 0.89572823517319355)
('IG:', 0.90007606126014994)
('IG:', 0.90432726898961868)
('IG:', 0.90432726898961868)
('IG:', 0.89997944290266219)
('IG:', 0.90007194984068239)
('IG:', 0.88693185322232504)
('IG:', 0.8913762976667694)
('IG:', 0.90007194984068

In [203]:
b = [traceAve_range, traceIG_range]
b.extend(tracesSingle_range)

py.iplot(b, filename='plot')

# Find intersecting features from each model

In [ ]:
_, ave = topAveDFSfeatures(0,102)
_, xg = topXGBoostfeatures(0,102)
_, union = unionDFSfeatures(27)

# Intersection indexes
intersect_3models = np.intersect1d(np.intersect1d(ave,union),xg).tolist()
print temp.shape

temp = inputX[:,intersect_3modsels]

# Accuracy rate based on the three models
np.mean(cross_val_score(svm, temp, inputY, cv=5))

In [169]:
X = np.random.rand(5,1)
y = np.array([0,1,1,0,1])

In [170]:
from sklearn.mixture import GMM

gmm = GMM(n_components=2)
gmm.fit(X)

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=2, n_init=1, n_iter=100, params='wmc', random_state=None,
  thresh=None, tol=0.001, verbose=0)

In [174]:
X

array([[ 0.42880099],
       [ 0.68039362],
       [ 0.52554149],
       [ 0.42296701],
       [ 0.33023037]])

In [175]:
gmm.fit_predict(X)

array([1, 0, 1, 1, 1])

In [192]:
def topIGfeatures(a,b):
    indexes_IG = np.argsort(ig)[::-1][a:b]
    return inputX[:, indexes_IG], indexes_IG

In [284]:
ig_cr[index_best]

array([  1.10895574e-01,   1.31042863e-01,   6.21529880e-03,
         2.52163416e-03,   2.41009417e-02,   1.61356490e-02,
         2.99863237e-03,   3.11532169e-02,   2.04260221e-01,
         1.78802492e-02,   9.52907904e-02,   8.16887351e-02,
         1.85355314e-01,   1.68254057e-01,   8.34945828e-03,
         1.11561634e-01,   1.39311341e-02,   6.99954995e-02,
         1.37771559e-03,   8.14885137e-02,   8.23198355e-02,
         1.36470745e-02,   2.36266777e-01,   6.31690827e-02,
         6.10165887e-02,   5.35009072e-02,   2.69106723e-04,
         2.52163416e-03,   1.78363600e-01,   3.27967201e-04,
         1.24932205e-01,   1.07417593e-01,   1.36867044e-03,
         8.14885137e-02,   1.11000438e-03,   5.12229034e-06,
         3.59021146e-03,   6.40510702e-02,   8.99896901e-04,
         3.22713229e-02,   8.99896901e-04,   2.15449993e-01,
         2.53409781e-03,   8.33308954e-03,   1.97232973e-04,
         1.66453792e-01,   3.27860052e-03,   2.47227796e-01,
         8.92561520e-03,

In [288]:
index_best

[779,
 1205,
 1295,
 1455,
 2090,
 2303,
 2468,
 2485,
 2838,
 3155,
 3245,
 3400,
 3477,
 3528,
 4006,
 4163,
 4383,
 4799,
 4927,
 5031,
 5336,
 5470,
 5674,
 5744,
 5817,
 6066,
 6966,
 7189,
 7441,
 7941,
 8017,
 8347,
 8791,
 9509,
 9942,
 10262,
 10443,
 10495,
 10723,
 10724,
 10802,
 10870,
 10887,
 11284,
 11432,
 11487,
 11598,
 11649,
 11783,
 11818,
 12195,
 12358,
 12844,
 12893,
 12996,
 13135,
 13868,
 13993,
 14109,
 14813,
 15570,
 15597,
 15867,
 16249,
 16482,
 17814,
 17901,
 18347,
 19271,
 19562,
 19688,
 20012,
 20258,
 20262,
 20587,
 20677,
 21291,
 21513,
 21780,
 21861,
 22238,
 22335,
 22459,
 22543,
 22731,
 22856,
 23379,
 23531,
 23703,
 23950,
 24301,
 24364,
 25119,
 25131,
 25239,
 25447,
 25712,
 25724,
 25792,
 25871,
 25950,
 26162,
 26365,
 26692,
 26790,
 26920,
 26931,
 27448,
 27591,
 27908,
 28272,
 29483,
 29633,
 29649,
 29714,
 29888,
 30397,
 30578,
 31170,
 31469,
 32389,
 32828,
 32870,
 32889,
 33226,
 33241,
 33751,
 34518,
 34679,
 348

In [287]:
np.argsort(ig_cr)

array([ 1290, 35575, 25522, ..., 38759, 16235, 21513])